<a href="https://colab.research.google.com/github/PardhivAryan/algo-trading-system/blob/main/Algo_Trading_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ------------------  Algo-Trading Strategy with Machine Learning & Google Sheets Logging -----------------------------------------

#  Importing reqired packages
import yfinance as yf # It is used for getting stock prices
import pandas as pd
import numpy as np
import gspread # It is used to connect with google sheet
from oauth2client.service_account import ServiceAccountCredentials
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import logging # It is used to write down important messages while it runs, so you can understand what it's doing or what went wrong.
import importlib

#  Logging Configuration
importlib.reload(logging) # flush any old settings
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True ) # This sets the default rules for how logging should work throughout
# the program .INFO shows lowest to highest are which are DEBUG, INFO, WARNING, ERROR, CRITICAL. and prints the time, level and message

#  Google Sheets Authentication
from google.colab import files
uploaded = files.upload()  # Upload the JSON file
json_filename = list(uploaded.keys())[0] # After uploading, the file is saved into json_filename and gets the name of the uploaded file so we can use it in the next
# step

scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"] # These links are permission scopes First one: read/write Google Sheets.
# Second one: read/write Google Drive files.
creds = ServiceAccountCredentials.from_json_keyfile_name(json_filename, scope) # uses the uploaded JSON file to create credentials the credentials access Google Sheets and Drive
client = gspread.authorize(creds) # logs us into Google Sheets using the credentials we just created.Now we can Open a sheet,Add rows, Read data, Clear a tab

# Connects to created Google Sheet with tabs 'Trade Log' and 'Summary'
sheet = client.open("Algo Trading Log")
log_tab = sheet.worksheet("Trade Log")
summary_tab = sheet.worksheet("Summary")

#  Fetch Historical Stock Data
def fetch_stock_data(ticker: str, period: str = '6mo') -> pd.DataFrame:
    data = yf.download(ticker, period=period, auto_adjust=False, progress=False) # It uses the yfinance library to get the stock’s historical data such as stock name like
    # "RELIANCE.NS", "AAPL", etc. and get 6 months of data. False means Don’t change the raw prices for dividends or stock splits. store resultant data into variable data
    return data.dropna()

#  Calculate Technical Indicators
def add_indicators(df: pd.DataFrame) -> pd.DataFrame: # Takes a stock price table (df) and adds three indicators: RSI, 20-day Moving Average (20DMA), 50-day Moving Average (50DMA)
    delta = df['Close'].diff() # Take the difference in closing prices between each day and the previous day.
    gain = delta.clip(lower=0) # keep only positive numbers
    loss = -delta.clip(upper=0) # keep only negative numbers, then flip sign
    avg_gain = gain.rolling(14).mean() # Calculate the average of the last 14 day's gains
    avg_loss = loss.rolling(14).mean() # Calculate the average of the last 14 day's losses.
    rs = avg_gain / avg_loss
    df['RSI'] = 100 - (100 / (1 + rs)) # RSI tells you if a stock is overbought or oversold:
    df['20DMA'] = df['Close'].rolling(20).mean() # These are average closing prices over the last 20 and 50 days. They smooth out the ups and downs.
    df['50DMA'] = df['Close'].rolling(50).mean() # These are average closing prices over the last 20 and 50 days. They smooth out the ups and downs.
    return df

#  Generate Buy Signals
def generate_signals(df): # This function looks at the table of stock prices and indicators and says which days look like good buying opportunities.
    signals = []
    for i in range(50, len(df)): # We go from day 50 to the end of the table.
        if df["RSI"].iloc[i] < 30 and df["20DMA"].iloc[i] > df["50DMA"].iloc[i]: #
            signals.append((df.index[i], df['Close'].iloc[i], 'BUY')) # Here we store date price and action
    return signals

#  Backtest Strategy
def simulate_trades(signals):
    if len(signals) < 2:  # It needs at least a buy and a sell
        return []
    trades = []
    for i in range(len(signals) - 1): # We will buy one the signal day and some random day we will buy again by selling the first bought one
        buy_date, buy_price, _ = signals[i] # when we bought, price when we bought
        sell_date, sell_price, _ = signals[i + 1] # when we sold, price when we sold
        pnl = sell_price - buy_price
        trades.append([ str(buy_date.date()),
                       float(buy_price),
                       float(sell_price),
                       float(pnl)]) # We will display the trade
    return trades

#  Log Trades to Google Sheets
def log_to_google_sheets(trades):
    log_tab.clear()
    log_tab.append_row(["Buy Date", "Buy Price", "Sell Price", "PnL"])
    for trade in trades: # Loop through the list of trades and write each one as a row in the Google Sheet.
        log_tab.append_row(trade)

    win_trades = [t for t in trades if t[3] > 0] # Here, t[3] is the PnL (Profit and Loss), We collect only the trades where PnL > 0 — i.e., winning trades.
    summary_tab.clear()
    summary_tab.append_row(["Total Trades", "Wins", "Win Ratio"])
    summary_tab.append_row([len(trades), # Total trades made
                            len(win_trades), # Trades that made profit
                            round(len(win_trades) / len(trades), 2) if trades else 0]) # f no trades, show win ratio as 0 to avoid division by zero.

#  ML-Based Next-Day Movement Prediction
def predict_with_ml(df): # Builds a machine learning model (using a decision tree), Predicts if the stock price will go up or down tomorrow
    df = df.dropna().copy() # Remove rows that have missing (NaN) values."
    df['MACD'] = df['Close'].ewm(span=12).mean() - df['Close'].ewm(span=26).mean() # Create a new column called MACD, MACD = difference between: 12-day EMA, 26-day EMA
    # MACD goes up → stock price may go up, MACD goes down → stock price may go down.
    df['Target'] = np.where(df['Close'].shift(-1) > df['Close'], 1, 0) # For each row, we check: Will the closing price tomorrow be higher than today. If yes → 1, no → 0
    features = df[['RSI', 'MACD', 'Volume']]
    labels = df['Target']
    # Split into Training and Test Data
    X_train, y_train = features[:-1], labels[:-1] # We use all rows except the last to train the model
    X_test, y_test = features[-1:], labels[-1:] # And the last row to test the prediction
    # Train a Decision Tree Model
    model = DecisionTreeClassifier() # Create a simple decision tree and teach it using the training data.
    model.fit(X_train, y_train) # The model learns  If RSI is low, MACD is positive, and volume is high, maybe the price goes up.
    prediction = model.predict(X_test) # Predict whether the stock price will go UP or DOWN tomorrow.
    # Measure Accuracy
    accuracy = accuracy_score(y_test, prediction) # Check if the model’s prediction was correct. If it guessed right → accuracy = 1.0, If wrong → accuracy = 0.0
    return prediction[0], accuracy

#  Run the Full Pipeline
tickers = ["INFY.NS", "RELIANCE.NS", "TCS.NS"]

for ticker in tickers:
    try:
        logging.info(f"Running analysis for {ticker}")

        data = fetch_stock_data(ticker) # It downloads the last 6 months of stock prices for the current ticker
        data = add_indicators(data) # This adds RSI, 20DMA, and 50DMA columns to the data.
        signals = generate_signals(data) # It checks the data and says based on RSI and moving averages, where should we BUY
        trades = simulate_trades(signals) # This pairs each two signals into a buy/sell trade and calculates profit/loss (PnL).
        log_to_google_sheets(trades) # This clears the sheet and writes all the trades + a summary (total trades, win ratio, etc.) to Google Sheets.

        direction, accuracy = predict_with_ml(data)
        logging.info(f"Logged {len(trades)} trades to Google Sheets")
        logging.info(f"ML Prediction for {ticker}: {'UP' if direction else 'DOWN'} "
                     f"with accuracy {accuracy:.2f}")

    except Exception as error:
        logging.error(f"Failed to process {ticker}: {error}")

Saving credentials.json to credentials (9).json


2025-06-25 15:38:02,625 - INFO - Running analysis for INFY.NS
2025-06-25 15:38:04,332 - INFO - Logged 0 trades to Google Sheets
2025-06-25 15:38:04,333 - INFO - ML Prediction for INFY.NS: UP with accuracy 0.00
2025-06-25 15:38:04,335 - INFO - Running analysis for RELIANCE.NS
2025-06-25 15:38:05,304 - INFO - Logged 0 trades to Google Sheets
2025-06-25 15:38:05,305 - INFO - ML Prediction for RELIANCE.NS: DOWN with accuracy 1.00
2025-06-25 15:38:05,307 - INFO - Running analysis for TCS.NS
/tmp/ipython-input-13-286150320.py:72: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(buy_price),
/tmp/ipython-input-13-286150320.py:73: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(sell_price),
/tmp/ipython-input-13-286150320.py:74: FutureWarning: Calling float on a single element Series is deprec